In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import time
from tqdm import trange
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import gym
import pybullet as p
import stage.envs
from stage.tasks.kuka import KukaPETS
from stage.utils.nn import use_gpu
use_gpu()

In [4]:
savepath = parentdir + '/data/kuka/'

In [5]:
kuka = KukaPETS(render=False, 
                   action_parameterization='pd')
nq, nv, nu, nx, na = kuka.nq, kuka.nv, kuka.nu, kuka.nx, kuka.na

In [6]:
_ = kuka.learn(30, verbose=True)
kuka.save_training_data(savepath+'data_pd_reg')

Network training: 100%|██████████| 10/10 [00:00<00:00, 26.06epoch(s)/s, Training loss MSE=20.452768]


Iteration:  0
avg. decision time:  0.004569455782572429
obs. reward:  -638.61304
act. reward:  -2.0568187
[tensor([0.0142, 0.0142, 0.0141, 0.0141, 0.0141, 0.0141, 0.0141, 0.0141, 0.0140,
        0.0140])]
[tensor([0.0138, 0.0138, 0.0138, 0.0137, 0.0137, 0.0137, 0.0137, 0.0137, 0.0136,
        0.0136])]
[tensor([0.0136, 0.0136, 0.0136, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135,
        0.0134])]
[tensor([0.0143, 0.0142, 0.0142, 0.0142, 0.0141, 0.0141, 0.0141, 0.0141, 0.0140,
        0.0139])]
[tensor([0.0140, 0.0140, 0.0140, 0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0138,
        0.0138])]
[tensor([0.0136, 0.0136, 0.0136, 0.0136, 0.0135, 0.0135, 0.0135, 0.0135, 0.0134,
        0.0134])]
[tensor([0.0191, 0.0185, 0.0181, 0.0180, 0.0179, 0.0177, 0.0174, 0.0174, 0.0166,
        0.0165])]
[tensor([0.0142, 0.0142, 0.0141, 0.0141, 0.0141, 0.0141, 0.0140, 0.0140, 0.0140,
        0.0140])]
[tensor([0.0138, 0.0137, 0.0137, 0.0137, 0.0137, 0.0137, 0.0137, 0.0137, 0.0137,
        0.0136])]
[te

[tensor([0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0133, 0.0133, 0.0133,
        0.0133])]
[tensor([0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0134, 0.0134, 0.0134,
        0.0134])]
[tensor([0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0130,
        0.0130])]
[tensor([0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0130,
        0.0130])]
[tensor([0.0131, 0.0131, 0.0131, 0.0131, 0.0130, 0.0130, 0.0130, 0.0130, 0.0130,
        0.0130])]
[tensor([0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134,
        0.0133])]
[tensor([0.0137, 0.0137, 0.0136, 0.0136, 0.0136, 0.0136, 0.0136, 0.0136, 0.0136,
        0.0136])]
[tensor([0.0132, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131, 0.0131,
        0.0131])]
[tensor([0.0135, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134, 0.0134,
        0.0134])]
[tensor([0.0136, 0.0136, 0.0136, 0.0135, 0.0135, 0.0135, 0.0135, 0.0134, 0.0134,
        0.0134])]
[tensor([0

Network training:  20%|██        | 2/10 [00:00<00:00, 11.66epoch(s)/s, Training loss MSE=21.231134]

[tensor([0.0133, 0.0133, 0.0133, 0.0133, 0.0133, 0.0133, 0.0132, 0.0132, 0.0132,
        0.0132])]


...

[tensor([0.0290, 0.0252, 0.0226, 0.0220, 0.0213, 0.0209, 0.0209, 0.0204, 0.0197,
        0.0182])]
[tensor([0.0569, 0.0325, 0.0250, 0.0229, 0.0227, 0.0223, 0.0218, 0.0213, 0.0196,
        0.0193])]
[tensor([0.0399, 0.0398, 0.0394, 0.0365, 0.0350, 0.0325, 0.0312, 0.0305, 0.0256,
        0.0215])]
[tensor([0.0540, 0.0507, 0.0457, 0.0420, 0.0337, 0.0331, 0.0313, 0.0270, 0.0263,
        0.0227])]
[tensor([0.0896, 0.0659, 0.0611, 0.0320, 0.0254, 0.0230, 0.0225, 0.0217, 0.0176,
        0.0171])]
[tensor([0.0725, 0.0343, 0.0319, 0.0298, 0.0288, 0.0273, 0.0264, 0.0245, 0.0244,
        0.0240])]
[tensor([0.0824, 0.0340, 0.0332, 0.0317, 0.0303, 0.0264, 0.0257, 0.0242, 0.0235,
        0.0201])]
[tensor([0.0536, 0.0318, 0.0275, 0.0263, 0.0239, 0.0233, 0.0224, 0.0213, 0.0189,
        0.0187])]
[tensor([0.0671, 0.0647, 0.0571, 0.0407, 0.0373, 0.0351, 0.0313, 0.0278, 0.0239,
        0.0199])]
[tensor([0.0383, 0.0377, 0.0325, 0.0296, 0.0292, 0.0286, 0.0268, 0.0241, 0.0207,
        0.0200])]
[tensor([0

Network training:   0%|          | 0/10 [00:00<?, ?epoch(s)/s]

[tensor([0.0392, 0.0306, 0.0283, 0.0267, 0.0256, 0.0231, 0.0225, 0.0217, 0.0209,
        0.0204])]


Network training: 100%|██████████| 10/10 [00:16<00:00,  1.63s/epoch(s), Training loss MSE=1.2745284]

Iteration:  29
avg. decision time:  0.20015802224477133
obs. reward:  -142.89333
act. reward:  -0.04340982


In [7]:
traj, log = kuka.perform(kuka.goal)
act_seq = traj[:, 14:28]
initial_obs = traj[0, :14]
final_obs = traj[-1, :14]

avg. decision time:  0.12732937335968017
obs. reward:  -236.11569
act. reward:  -0.70518595


In [ ]:
n_sample = 50
traj_pred = kuka.dynamics.unroll(initial_obs, 
                                 act_seq, 
                                 n_sample)

predicted_err = torch.norm(traj_pred[-1, :, :]-final_obs.expand(n_sample, -1), p=2, dim=1)
print (predicted_err.mean())
print (predicted_err.std())

In [ ]:
traj_pred_mean = torch.mean(traj_pred, dim=1)
traj_pred_std = torch.std(traj_pred, dim=1)
traj_pred_mean = traj_pred_mean.detach().cpu().numpy()
traj_pred_std = traj_pred_std.detach().cpu().numpy()
traj = traj.detach().cpu().numpy()

In [ ]:
font = {'family' : 'serif',
        'size'   : 22}

matplotlib.rc('font', **font)
np.set_printoptions(precision=3, linewidth=200, suppress=True)

In [ ]:
# traj_pred_mean = np.load('traj_pred_mean.npy')
# traj_pred_std = np.load('traj_pred_std.npy')
# traj = np.load('traj.npy')

In [ ]:
d = 1
dt = 0.01
task_horizon = 150
fig, ax = plt.subplots(d, figsize=(10, d * 6))
t = np.arange(0.0, task_horizon*dt, dt)

i = 1
desired = kuka.step_cost.desired[i].detach().cpu().numpy()*np.ones(len(t))

ax.plot(t, traj_pred_mean[:, i], lw=4, color='b', label='predicted mean')
ax.plot(t, traj[:, i], lw=4, ls='-.', color='orange', label='actual')

ax.plot(t, desired, lw=2, color='k', ls='-.', label='desired')

lb = traj_pred_mean - traj_pred_std
ub = traj_pred_mean + traj_pred_std
ax.fill_between(t, lb[:, i], ub[:, i], facecolor='blue',
            alpha=0.2, label='one-std region')
_ = ax.grid()
_ = ax.set_ylabel('Joint angle [rad]')
_ = ax.set_xlabel('Time [s]')
# _ = ax.set_xlim([0,0.5])
_ = ax.set_ylim([-3.2, 3.2])

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
             ncol=2, fancybox=True, shadow=True)
# for i in range(d):
#     ax[i].plot(t, traj[:, i], lw=4, color='orange', label='actual')
#     ax[i].plot(t, traj_pred_mean[:, i], lw=4, color='b', label='predicted mean')
#     lb = traj_pred_mean - traj_pred_std
#     ub = traj_pred_mean + traj_pred_std
#     ax[i].fill_between(t, lb[:, i], ub[:, i], facecolor='blue',
#                 alpha=0.2)
# fig.savefig('prediction_with_reg_150steps' + '.png', bbox_inches='tight')